## Audit Your Customer Journey Analytics Implementation with Python

This script is designed to help you audit your Customer Journey Analytics (CJA) implementation by generating a comprehensive spreadsheet. This spreadsheet will list all metrics and dimensions available in your selected CJA Data View, providing an invaluable resource akin to a Solution Design Reference (SDR) document in traditional Adobe Analytics. This tool is especially useful for identifying what's been configured in CJA and ensuring your analytics setup aligns with your measurement strategy.

### What the Script Does

1. **Initial Setup**: Utilizes the `cjapy` library to interact with your CJA account, requiring a configuration file for authentication.
2. **Data View Selection**: Targets a specific CJA Data View by its ID for auditing.
3. **Retrieves Metrics and Dimensions**: Fetches all metrics and dimensions configured in the selected Data View.
4. **Generates a Detailed Spreadsheet**: Outputs an Excel file containing:
    - A sheet for DataView details.
    - A sheet listing all Metrics, formatted for readability.
    - A sheet listing all Dimensions, also formatted for readability.

### Pre-requisites

- Python installed on your system.
- The `cjapy` library installed. You can install it using `pip install cjapy`.
- An existing `python_config.json` configuration file with your CJA credentials.
- Pandas library installed for data manipulation and Excel file generation.

### Usage

1. Ensure you have your `python_config.json` file ready and correctly configured.
2. Replace `"dv_62ba17d5a5d7845496f5fb4d"` with your actual Data View ID.
3. Run the script. It will automatically generate an Excel file named `CJA_DataView_[DataViewName]_[DataViewID]_SDR.xlsx`.
4. Open the generated Excel file to review your CJA implementation details.

### Use Cases

- **Comprehensive Audit**: Quickly understand the breadth and depth of your CJA setup, including what metrics and dimensions are available for analysis.
- **Implementation Verification**: Ensure that your CJA implementation matches your initial planning documents or SDR.
- **Onboarding**: Assist new team members in getting up to speed with the existing CJA setup by providing them with a detailed view of the data structure.
- **Documentation**: Automatically generate part of your project documentation that outlines the analytical capabilities enabled in CJA.

This script automates what could be a time-consuming manual process, allowing you to spend more time on analysis and less on administration.

In [1]:
import cjapy
import pandas as pd

# Instantiate cjapy
cjapy.importConfigFile("python_config.json")
cja = cjapy.CJA()

# Set the Data View id we want into a variable
data_view = "dv_62ba17d5a5d7845496f5fb4d"

# Grab all available CJA metrics
metrics = cja.getMetrics(data_view, inclType=True, full=True)

# Grab all available CJA dimensions
dimensions = cja.getDimensions(data_view, inclType=True, full=True)

# Perform a CJA data view lookup using getDataView module
lookup_data = cja.getDataView(data_view)

# Ensure all values are iterable
for key, value in lookup_data.items():
    if not isinstance(value, (list, tuple)):
        lookup_data[key] = [value]

# Ensure all arrays have the same length
max_length = max(len(value) for value in lookup_data.values())
for key, value in lookup_data.items():
    if len(value) < max_length:
        lookup_data[key] += [None] * (max_length - len(value))

# Convert dictionary to DataFrame
lookup_df = pd.DataFrame(lookup_data)

# Create an Excel writer object with CJA data view name and data view id included in the file name
excel_file_name = 'CJA_DataView_' + str(lookup_data["name"]) + '_' + str(data_view) + '_SDR.xlsx'
with pd.ExcelWriter(excel_file_name, engine='xlsxwriter') as writer:


    # Write lookup data to first tab
    lookup_df.to_excel(writer, sheet_name='DataView', index=False)

    # Write metrics data to the second tab
    metrics.to_excel(writer, sheet_name='Metrics', index=False)
    
    # Get the workbook and worksheet objects
    workbook = writer.book
    worksheet = writer.sheets['Metrics']
    
    # Define formats for grey and white rows
    grey_format = workbook.add_format({'bg_color': '#F2F2F2'})
    white_format = workbook.add_format({'bg_color': '#FFFFFF'})
    
    # Apply grey and white formatting to Metrics sheet
    for idx, row in enumerate(metrics.iterrows()):
        if idx % 2 == 0:
            worksheet.set_row(idx+1, cell_format=grey_format)
        else:
            worksheet.set_row(idx+1, cell_format=white_format)
    
    # Write dimensions data to the third tab
    dimensions.to_excel(writer, sheet_name='Dimensions', index=False)
    
    # Get the worksheet object for Dimensions sheet
    worksheet = writer.sheets['Dimensions']
    
    # Apply grey and white formatting to Dimensions sheet
    for idx, row in enumerate(dimensions.iterrows()):
        if idx % 2 == 0:
            worksheet.set_row(idx+1, cell_format=grey_format)
        else:
            worksheet.set_row(idx+1, cell_format=white_format)
